In [1]:
! pip install transformers

In [2]:
import torch
import pandas as pd
import tensorflow as tf
import time
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
import random
import os
from torch import optim

# 경고 메시지 무시 설정
warnings.filterwarnings("ignore")

In [3]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 가능한 디바이스: {device}")

사용 가능한 디바이스: cuda


In [4]:
device

device(type='cuda')

In [8]:
file_path = '/content/황금_트윗추가.csv'
data = pd.read_csv(file_path, encoding='CP949')

In [ ]:
stop_words = set(stopwords.words('english'))

# 추가할 불용어 목록
additional_stopwords = {'your', 'custom', 'stopword'}
stop_words.update(additional_stopwords)

In [ ]:
def remove_stopwords(sentence):
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    return ' '.join(filtered_sentence)

In [ ]:
# 'text' 열에 불용어 제거 함수를 적용합니다. 'text'는 예시로 사용된 열 이름이며, 실제 데이터에 따라 다를 수 있습니다.
data['Dialogue'] = data['Dialogue1'].apply(remove_stopwords)

In [ ]:
data

,Dialogue,Emotion
0,"['reported', 'to', 'school', 'violence', 'comm...",기쁨(Joy)
1,"['im', 'worried', 'daughter', 'wont', 'able', ...",걱정(Worry)
2,"['daughter', 'keeps', 'not', 'tidying', 'up', ...",걱정(Worry)
3,"['im', 'angry', 'because', 'got', 'into', 'fig...",분노(Anger)
4,"['things', 'changed', 'after', 'children', 'go...",분노(Anger)
...,...,...
42541,"['Im', 'left', 'incredulous', 'anyone', 'see',...",슬픔(Sadness)
42542,"['This', 'movie', 'inspires', 'feelings', 'of'...",슬픔(Sadness)
42543,"['This', 'one', 'of', 'few', 'movies', 'just',...",슬픔(Sadness)
42544,"['In', 'short', 'under', 'hour', 'feature', 'a...",슬픔(Sadness)


In [9]:
# Emotion 컬럼의 각각의 종류별 개수 구하기
emotion_counts = data['Emotion'].value_counts()
emotion_counts

슬픔(Sadness)    11916
기쁨(Joy)        10500
사랑(Love)        8811
걱정(Worry)       8464
중립(Neutral)     8326
분노(Anger)       8290
Name: Emotion, dtype: int64

In [10]:
data['Dialogue'] = data['Dialogue'].str.lower()

In [11]:
data

,Dialogue,Emotion
0,i reported it to the school violence committee...,기쁨(Joy)
1,im worried that my daughter wont be able to fi...,걱정(Worry)
2,my daughter keeps not tidying up her room so i...,걱정(Worry)
3,im angry because i got into a fight with my cl...,분노(Anger)
4,things changed after my children got married i...,분노(Anger)
...,...,...
56302,tambor and clayburgh make an appealing couple ...,기쁨(Joy)
56303,the best thing about the movie is its personab...,기쁨(Joy)
56304,here is a divine monument to a single man 's s...,기쁨(Joy)
56305,"painful , horrifying and oppressively tragic ,...",기쁨(Joy)


In [12]:
# "Dialogue" 컬럼에서 중복된 항목을 확인합니다.
duplicates = data[data.duplicated(subset='Dialogue', keep=False)]

# 중복된 항목의 수와 몇몇 예시를 보여줍니다.
duplicate_count = duplicates.shape[0]
duplicate_examples = duplicates.head()

duplicate_count, duplicate_examples

(1435,
                                                Dialogue    Emotion
 0     i reported it to the school violence committee...    기쁨(Joy)
 777   i have been exercising for a long time and am ...    기쁨(Joy)
 2028  wouldnt it be better than how it looks now wit...  걱정(Worry)
 2029  why did they make such decision all of a sudde...  걱정(Worry)
 2030  i dont understand why they keep doing this eve...  분노(Anger))

In [13]:
# "Dialogue" 컬럼에서 중복된 항목을 제거합니다. keep='first'는 첫 번째 항목을 제외하고 중복을 제거합니다.
data = data.drop_duplicates(subset='Dialogue', keep='first')

# 중복 제거 후 남은 데이터의 개수를 확인합니다.
remaining_data_count = data.shape[0]

remaining_data_count

55588

In [14]:
# 'Dialogue' 컬럼에서 특수 문자 제거
data['Dialogue'] = data['Dialogue'].str.replace('[^A-Za-z0-9가-힣 ]', '', regex=True)

In [ ]:
data.head()

,Dialogue,Emotion
0,reported to school violence committee so feel ...,기쁨(Joy)
1,im worried daughter wont able to find job year...,걱정(Worry)
2,daughter keeps not tidying up room so im reall...,걱정(Worry)
3,im angry because got into fight with classmate...,분노(Anger)
4,things changed after children got married if d...,분노(Anger)


In [15]:
emotion_counts = data['Emotion'].value_counts()
emotion_counts

슬픔(Sadness)    11896
기쁨(Joy)        10296
사랑(Love)        8618
걱정(Worry)       8329
중립(Neutral)     8325
분노(Anger)       8124
Name: Emotion, dtype: int64

In [16]:
data.count()

Dialogue    55588
Emotion     55588
dtype: int64

In [ ]:
data

,Dialogue,Emotion
0,reported to school violence committee so feel ...,기쁨(Joy)
1,im worried daughter wont able to find job year...,걱정(Worry)
2,daughter keeps not tidying up room so im reall...,걱정(Worry)
3,im angry because got into fight with classmate...,분노(Anger)
4,things changed after children got married if d...,분노(Anger)
...,...,...
42541,im left incredulous anyone see film as anythin...,슬픔(Sadness)
42542,this movie inspires feelings of disgust in not...,슬픔(Sadness)
42543,this one of few movies just so dreadful i coul...,슬픔(Sadness)
42544,in short under hour feature all narrative whic...,슬픔(Sadness)


In [ ]:
# # CSV 파일로 저장
# file_path = "./data/황금.csv"
# data.to_csv(file_path, index=False, encoding="utf-8")

In [17]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BERT tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 토큰화
tokenized_data = tokenizer(
    data['Dialogue'].tolist(),
    add_special_tokens=True,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

# 입력 데이터 준비
input_ids = tokenized_data['input_ids']
attention_mask = tokenized_data['attention_mask']

# 레이블 인코딩
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(data['Emotion'])
labels = torch.tensor(labels_encoded, dtype=torch.long)

# 데이터셋 분할
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(
    input_ids[:len(data)],
    labels,
    test_size=0.2,
    random_state=42
)
train_attention_mask, val_attention_mask, _, _ = train_test_split(
    attention_mask[:len(data)],
    labels,
    test_size=0.2,
    random_state=42
)

# 데이터 로더 생성
train_data = TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

val_data = TensorDataset(val_input_ids, val_attention_mask, val_labels)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=False)

# BERT 모델 초기화
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model = model.to(device)

# 옵티마이저 설정
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# 얼리스탑 조기 종료 설정
early_stopping_patience = 2
best_val_loss = float('inf')
no_improvement_count = 0

# 에포크 학습 시간을 저장할 리스트
epoch_times = []

# 학습 설정
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    start_time = time.time()

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    end_time = time.time()
    avg_train_loss = total_loss / len(train_dataloader)
    epoch_time = end_time - start_time
    print(f'Epoch {epoch + 1}, Loss: {avg_train_loss}, Time: {epoch_time} seconds')

    # 검증 손실 계산
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_dataloader)

    # 얼리스탑 체크
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print(f'조기 종료됨. {epoch + 1} 에포크에서 멈춤.')
            break

print("야후! 끝나써여!")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.7463725209102142, Time: 1240.8711607456207 seconds
Epoch 2, Loss: 0.4241846632480407, Time: 1239.6476006507874 seconds
Epoch 3, Loss: 0.30994712469442715, Time: 1240.4113149642944 seconds
Epoch 4, Loss: 0.2319695186395895, Time: 1239.0753059387207 seconds
조기 종료됨. 4 에포크에서 멈춤.
야후! 끝나써여!


In [18]:
# 학습 완료 후 정확도 검증
model.eval()
all_val_labels = []
all_val_preds = []

with torch.no_grad():
    for batch in val_dataloader:
        batch = [item.to(device) for item in batch]
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_val_labels.extend(labels.cpu().numpy())
        all_val_preds.extend(preds.cpu().numpy())

# 검증 데이터의 정확도 계산
accuracy = accuracy_score(all_val_labels, all_val_preds)
print(f'검증 데이터 정확도: {accuracy}')

검증 데이터 정확도: 0.827217125382263


In [19]:
# 모델 저장 경로 지정
model_save_path = "/content/model/bert_gold_model6.pth"

# 필요한 경우 디렉토리 생성
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# 모델의 상태 사전 저장
torch.save(model.state_dict(), model_save_path)
print(f"모델 저장 완료! 저장 경로: {model_save_path}")

모델 저장 완료! 저장 경로: /content/model/bert_gold_model6.pth
